In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import pickle
from string import punctuation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

!pip install pymystem3
from pymystem3 import Mystem

nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
recipes=pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv',
                   converters={'tags': lambda x: x[1:-1].replace('"', '').replace("'", "").replace(" ", "").split(',')}
                   )

recipes["Text"] = recipes["name"] + ' ' + recipes["description"] + ' ' + recipes["steps"]
recipes = recipes.drop(['id', 'minutes','contributor_id', 'nutrition', 'n_steps', 'n_ingredients', 'submitted', 'name', 'description', 'steps', 'ingredients'], axis=1)
recipes.head()

In [ ]:
stop_list = ['ThrowtheultimatefiestawiththissopaipillasrecipefromFood.com.', 
               'preparation', 'time-to-make', 'course', 'cuisine', 'low-in-something',
               'equipment', 'number-of-servings', 'healthy-2', 'free-of-something' ,'technique',
               'high-in-something', 'high-in-something-diabetic-friendly', 
               'less_thansql:name_topics_of_recipegreater_than', '15-minutes-or-less', '30-minutes-or-less', '60-minutes-or-less', 
               '4-hours-or-less', '3-steps-or-less', '5-ingredients-or-less', 'very-low-carbs', '1-day-or-more', 'copycat',
               'oamc-freezer-make-ahead', 'heirloom-historical',
              ]

recipes['tags'] = recipes['tags'].apply(
    lambda row: [val for val in row if val not in stop_list]
)

In [ ]:
def clean_text(text):
    text = re.sub("\'", "", text)
    text = re.sub("," , " ", text)
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower() 
    return text

recipes.Text = recipes.Text.fillna(" no recipe ")
recipes.Text = recipes.Text.apply(lambda x: clean_text(x))

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

recipes.Text = recipes.Text.apply(lambda x: remove_stopwords(x))

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    lemmatized_text=[]
    sentence_words = nltk.word_tokenize(text)
    for word in sentence_words:
        lemmatized_text.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = ' '.join(lemmatized_text)
    return lemmatized_text

recipes.Text = recipes.Text.apply(lambda x: lemmatize_text(x))

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(recipes['tags'])

y = multilabel_binarizer.transform(recipes['tags'])

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

xtrain, xval, ytrain, yval = train_test_split(recipes.Text, y, test_size=0.2, random_state=9)
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

lr = LogisticRegression(max_iter = 1000, class_weight='balanced')
clf = OneVsRestClassifier(lr)
clf.fit(xtrain_tfidf, ytrain)

In [ ]:
y_pred = clf.predict(xval_tfidf)
f1_score(yval, y_pred, average="micro")

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q = lemmatize_text(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

infer_tags('fried chicken')

In [ ]:
model_filename = 'multilabel_model.sav'
pickle.dump(clf, open(model_filename, 'wb'))

tfidf = 'tfidf.sav'
pickle.dump(tfidf_vectorizer, open(tfidf, 'wb'))

model_filename = 'multilabel_binarizer.sav'
pickle.dump(multilabel_binarizer, open(model_filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('/kaggle/working/multilabel_model.sav', 'rb'))
tfidf_vect_loaded = pickle.load(open('/kaggle/working/tfidf.sav', 'rb'))
binarizer = pickle.load(open('/kaggle/working/multilabel_binarizer.sav', 'rb'))

def infer_tags2(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q = lemmatize_text(q)
    q_vec = tfidf_vect_loaded.transform([q])
    q_pred = loaded_model.predict(q_vec)
    return binarizer.inverse_transform(q_pred)
infer_tags2(" pizza ")